In [1]:
import tensorflow as tf
import numpy as np
import pandas as pd
import cv2
import os
#import confusion matrix
from sklearn.metrics import confusion_matrix
import matplotlib.pyplot as plt
import seaborn as sns



In [2]:
def load_images_from_folder(folder):
    images = []
    labels = []
    class_names = os.listdir(folder)
    # print(class_names)
    class_index = 0
    
    for class_name in class_names:
        class_folder = os.path.join(folder, class_name)
        if os.path.isdir(class_folder):
            for filename in os.listdir(class_folder):
                img_path = os.path.join(class_folder, filename)
                # Read the image using OpenCV
                image = cv2.imread(img_path)
                # Resize the image if needed
                # image = cv2.resize(image, (width, height))
                if image is not None:
                    images.append(image)
                    labels.append(class_index)
        class_index += 1
    
    return np.array(images), np.array(labels)


In [3]:
train_images, train_labels = load_images_from_folder('task1/train')
test_images, test_labels = load_images_from_folder('task1/test')
valid_images, valid_labels = load_images_from_folder('task1/valid')

print('Train images:', len(train_images))
print('Test images:', len(test_images))
print('Valid images:', len(valid_images))

Train images: 2000
Test images: 500
Valid images: 500


In [4]:
# class index to name
# 0 - beetle
# 1 - sloth
# 2 - pencil pouch
# 3 - xerox machine
# 4 - vase

In [5]:
print(train_images[0].shape)
# print(train_images[0])

(224, 224, 3)


In [6]:
train_images = train_images / 255.0
test_images = test_images / 255.0
valid_images = valid_images / 255.0

# print(train_images[0])

In [7]:
vgg_model = tf.keras.applications.VGG16(weights='imagenet', include_top=False, input_shape=(224, 224, 3))
vgg_model.trainable = False
# print(vgg_model.summary())
inception_model = tf.keras.applications.InceptionV3(weights='imagenet', include_top=False, input_shape=(224, 224, 3))
inception_model.trainable = False
# print(inception_model.summary())

flatten_layer = tf.keras.layers.Flatten()

In [8]:
model_vgg = tf.keras.models.Sequential([
    vgg_model,
    flatten_layer,
    tf.keras.layers.Dense(100, activation='tanh'),
    tf.keras.layers.Dense(50, activation='tanh'),
    tf.keras.layers.Dense(5, activation='softmax')
])

model_inception = tf.keras.models.Sequential([
    inception_model,
    flatten_layer,
    tf.keras.layers.Dense(100, activation='tanh'),
    tf.keras.layers.Dense(50, activation='tanh'),
    tf.keras.layers.Dense(5, activation='softmax')
])

In [9]:
model_vgg.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
model_inception.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

In [10]:
model_vgg.fit(train_images, train_labels, epochs=3, validation_data=(valid_images, valid_labels))
model_inception.fit(train_images, train_labels, epochs=3, validation_data=(valid_images, valid_labels))

#as the validaition accuracy started to decrease after 3 epochs, we stop training the model to avoid overfitting


Epoch 1/3
63/63 [==============================] - 204s 3s/step - loss: 0.8854 - accuracy: 0.7570 - val_loss: 0.4073 - val_accuracy: 0.8940
Epoch 2/3
63/63 [==============================] - 210s 3s/step - loss: 0.2351 - accuracy: 0.9450 - val_loss: 0.3047 - val_accuracy: 0.9000
Epoch 3/3
63/63 [==============================] - 203s 3s/step - loss: 0.1141 - accuracy: 0.9795 - val_loss: 0.2858 - val_accuracy: 0.9060
Epoch 1/3
63/63 [==============================] - 46s 713ms/step - loss: 0.3168 - accuracy: 0.9245 - val_loss: 0.1050 - val_accuracy: 0.9820
Epoch 2/3
63/63 [==============================] - 44s 696ms/step - loss: 0.0812 - accuracy: 0.9860 - val_loss: 0.0925 - val_accuracy: 0.9820
Epoch 3/3
63/63 [==============================] - 42s 672ms/step - loss: 0.0674 - accuracy: 0.9860 - val_loss: 0.0785 - val_accuracy: 0.9860


In [11]:
#getting the test accuracy
vgg_test_loss, vgg_test_accuracy = model_vgg.evaluate(test_images, test_labels)
inception_test_loss, inception_test_accuracy = model_inception.evaluate(test_images, test_labels)

print('VGG test accuracy:', vgg_test_accuracy)
print('Inception test accuracy:', inception_test_accuracy)

16/16 [==============================] - 9s 560ms/step - loss: 0.1066 - accuracy: 0.9720
VGG test accuracy: 0.9259999990463257
Inception test accuracy: 0.972000002861023


In [21]:
# Generate predictions on training and test data
train_predictions_vgg = model_vgg.predict(train_images)
test_predictions_vgg = model_vgg.predict(test_images)
train_predictions_inception = model_inception.predict(train_images)
test_predictions_inception = model_inception.predict(test_images)



16/16 [==============================] - 8s 495ms/step


In [22]:
train_predictions_vgg = np.argmax(train_predictions_vgg, axis=1)
test_predictions_vgg = np.argmax(test_predictions_vgg, axis=1)
train_predictions_inception = np.argmax(train_predictions_inception, axis=1)
test_predictions_inception = np.argmax(test_predictions_inception, axis=1)

In [23]:
vgg_test_loss, vgg_test_accuracy = model_vgg.evaluate(test_images, test_labels)
inception_test_loss, inception_test_accuracy = model_inception.evaluate(test_images, test_labels)
vgg_train_loss, vgg_train_accuracy = model_vgg.evaluate(train_images, train_labels)
inception_train_loss, inception_train_accuracy = model_inception.evaluate(train_images, train_labels)

63/63 [==============================] - 87s 1s/step - loss: 0.0516 - accuracy: 0.9910


In [24]:
print(train_predictions_vgg[:10])

[0 0 0 0 0 0 0 0 0 0]


In [25]:
# Calculate confusion matrices for training and test data
train_confusion_matrix_vgg = confusion_matrix(train_labels, train_predictions_vgg)
test_confusion_matrix_vgg = confusion_matrix(test_labels, test_predictions_vgg)

# Calculate confusion matrices for training and test data
train_confusion_matrix_inception = confusion_matrix(train_labels, train_predictions_inception)
test_confusion_matrix_inception = confusion_matrix(test_labels, test_predictions_inception)

print("Train Confusion Matrix for VGGNet")
print(train_confusion_matrix_vgg)
print('Accuracy for train data using VGGNet: ',vgg_train_accuracy)
print("Test Confusion Matrix for VGGNet")
print(test_confusion_matrix_vgg)
print('Accuracy for test data using VGGNet: ',vgg_test_accuracy)

print("Train Confusion Matrix for Inception")
print(train_confusion_matrix_inception)
print('Accuracy for train data using Inception: ',inception_train_accuracy)
print("Test Confusion Matrix for Inception")
print(test_confusion_matrix_inception)
print('Accuracy for test data using Inception: ',inception_test_accuracy)

Train Confusion Matrix for VGGNet
[[396   0   3   1   0]
 [  0 390   1   1   8]
 [  4   0 393   0   3]
 [  1   1   0 394   4]
 [  0   0   0   1 399]]
Accuracy for train data using VGGNet:  0.9860000014305115
Test Confusion Matrix for VGGNet
[[93  0  6  1  0]
 [ 0 89  0  3  8]
 [ 7  0 93  0  0]
 [ 1  0  3 94  2]
 [ 1  0  1  4 94]]
Accuracy for test data using VGGNet:  0.9259999990463257
Train Confusion Matrix for Inception
[[398   0   0   2   0]
 [  0 397   0   0   3]
 [  5   0 390   2   3]
 [  0   0   0 399   1]
 [  0   0   0   2 398]]
Accuracy for train data using Inception:  0.9909999966621399
Test Confusion Matrix for Inception
[[98  0  0  0  2]
 [ 0 98  0  0  2]
 [ 4  1 92  0  3]
 [ 0  0  0 99  1]
 [ 0  0  0  1 99]]
Accuracy for test data using Inception:  0.972000002861023
